# Prompts

Prompt allow you to create System Messages with input variables, for example, this: SystemMessage(content="You are a helpful assistant that translates the English to Spanish.") English and Spanish may be dynamic. This can be archieved with templates

In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
TEMPLATE = """
You are a helpful assistant that translates the {input_language} to {output_language}
"""

In [3]:
from langchain.prompts.prompt import PromptTemplate

prompt_template = PromptTemplate.from_template(
    template=TEMPLATE
)
prompt_template.format(input_language="english", output_language="german")

'\nYou are a helpful assistant that translates the english to german\n'

In [4]:
prompt_template = PromptTemplate(template=TEMPLATE, input_variables=["input_language", "output_language"])
prompt_template.format(input_language="english", output_language="german")

'\nYou are a helpful assistant that translates the english to german\n'